In [1]:
from azure.ai.evaluation.simulator import Simulator

In [ ]:
# pip install wikipedia

In [3]:
import asyncio
from azure.identity import DefaultAzureCredential
import wikipedia
import os
from typing import List, Dict, Any, Optional
# Prepare the text to send to the simulator
wiki_search_term = "Leonardo da vinci"
wiki_title = wikipedia.search(wiki_search_term)[0]
wiki_page = wikipedia.page(wiki_title)
text = wiki_page.summary[:5000]

In [4]:
text

"Leonardo di ser Piero da Vinci (15 April 1452 – 2 May 1519) was an Italian polymath of the High Renaissance who was active as a painter, draughtsman, engineer, scientist, theorist, sculptor, and architect. While his fame initially rested on his achievements as a painter, he has also become known for his notebooks, in which he made drawings and notes on a variety of subjects, including anatomy, astronomy, botany, cartography, painting, and palaeontology. Leonardo is widely regarded to have been a genius who epitomised the Renaissance humanist ideal, and his collective works comprise a contribution to later generations of artists matched only by that of his younger contemporary Michelangelo.\nBorn out of wedlock to a successful notary and a lower-class woman in, or near, Vinci, he was educated in Florence by the Italian painter and sculptor Andrea del Verrocchio. He began his career in the city, but then spent much time in the service of Ludovico Sforza in Milan. Later, he worked in Flo

In [ ]:
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "type": "azure_openai"
}

In [12]:
async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    query = messages["messages"][0]["content"]

    formatted_response = {"content": query, "role": "assistant"}
    messages["messages"].append(formatted_response)
    return {
        "messages": messages["messages"],
        "stream": stream,
        "session_state": session_state,
        "context": context,
    }


In [24]:
#model_config = {
#        "azure_endpoint": "<your_azure_endpoint>",
#        "azure_deployment": "<deployment_name>"
#    }

# https://learn.microsoft.com/en-us/python/api/azure-ai-evaluation/azure.ai.evaluation.simulator.simulator?view=azure-python
simulator = Simulator(model_config=model_config)

outputs = await simulator(
    target=callback,
    text=text,
    tasks=[f"I am a 5 year old child wanting to know more about {wiki_search_term}", F"Syo una chica que quiere saber de {wiki_search_term}"],
    num_queries=2,  # Minimal number of queries
)

Generating: 100%|██████████████████████████████████████████████| 10/10 [00:18<00:00,  1.89s/message]


In [25]:
outputs

[{'messages': [{'role': 'user',
    'content': 'Can you tell me when Leonardo da Vinci was born?',
    'context': 'None'},
   {'role': 'assistant',
    'content': 'Can you tell me when Leonardo da Vinci was born?',
    'context': ''},
   {'role': 'user',
    'content': "Wow, that's cool! What did Leonardo da Vinci do?",
    'context': 'None'},
   {'role': 'assistant',
    'content': 'Can you tell me when Leonardo da Vinci was born?',
    'context': ''},
   {'role': 'user',
    'content': 'Leonardo da Vinci was a painter, inventor, and scientist! He painted the famous Mona Lisa and The Last Supper. He also designed flying machines and studied the human body. Do you want to know more about his paintings or inventions?',
    'context': 'None'},
   {'role': 'assistant',
    'content': 'Can you tell me when Leonardo da Vinci was born?',
    'context': ''},
   {'role': 'user',
    'content': 'I want to know more about his inventions! What kind of flying machines did he design?',
    'context